In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# Cargar el conjunto de datos
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(newsgroups.data, newsgroups.target, test_size=0.4, random_state=42)

# Dividir el conjunto de entrenamiento en subconjuntos con superposición
def create_overlapping_subsets(X, y, subset_size=1500, overlap=500, num_subsets=11):
    subsets_X = []
    subsets_y = []
    start = 0

    for i in range(num_subsets):
        end = start + subset_size if i < num_subsets - 1 else len(X)
        subsets_X.append(X[start:end])
        subsets_y.append(y[start:end])
        start = end - overlap if i < num_subsets - 1 else len(X)
    return subsets_X, subsets_y

subsets_X_train, subsets_y_train = create_overlapping_subsets(X_train, y_train)


In [3]:
[len(subsets_X_train[i]) for i in range(11)]

[1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1307]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear la matriz TF-IDF para cada subconjunto
tfidf_vectorizer = TfidfVectorizer(max_features=None)  # Sin limitar el número de características
tfidf_global = tfidf_vectorizer.fit_transform(X_train)
subsets_tfidf = [tfidf_vectorizer.transform(subset) for subset in subsets_X_train]


In [5]:
[subsets_tfidf[i].shape[1] for i in range(11)]

[96465, 96465, 96465, 96465, 96465, 96465, 96465, 96465, 96465, 96465, 96465]

In [6]:
# Import librería para simular SNN
from brian2 import *

start_scope()

# Asumiendo que tfidf_matrix ya está definida y es accesible
tfidf_matrix = subsets_tfidf[0]
M = tfidf_matrix.shape[1]  # Número de neuronas en la primera capa, basado en la cantidad de características TF-IDF
N = 1  # Neurona inhibitoria
alpha = 1.5  # Coeficiente de proporcionalidad para las tasas de disparo

# Parámetros del modelo LIF
tau = 100*ms
u_rest = -65*mV
u_exc = 0*mV
u_inh = -90*mV
u_th = -52*mV
t_ref = 3*ms
tau_e = 2*ms
tau_i = 2*ms

# Modelo de neurona LIF
eqs = '''
                du/dt = ((u_rest - u) + g_e*(u_exc - u) + g_i*(u_inh - u)) / tau : volt (unless refractory)
                dg_e/dt = -g_e / tau_e : 1
                dg_i/dt = -g_i / tau_i : 1
                '''

# Creación de grupos de neuronas
G = NeuronGroup(M, eqs, threshold='u > u_th', reset='u = u_rest', refractory=t_ref, method='euler')
I = NeuronGroup(N, eqs, threshold='u > u_th', reset='u = u_rest', refractory=t_ref, method='euler')

G.u = u_rest
I.u = u_rest

# Sinapsis
S_ei = Synapses(G, I, on_pre='g_e += 0.2')
S_ei.connect()

S_ie = Synapses(I, G, on_pre='g_i += 0.5')
S_ie.connect()

# Corrección: Renombrar A_pre y R_post para evitar el error
# Plasticidad STDP
eqs_plasticity = '''
                dA/dt = -A / tau_A : 1 (event-driven)
                dR/dt = -R / tau_R : 1 (event-driven)
                w : 1
                '''
on_pre = '''
                A += 1
                w = clip(w + (n * (A - (R + 0.1) * w)), 0, w_max)
                '''
on_post = '''
                R += 1
                '''

tau_A = 5*ms  # Constante de tiempo de traza presináptica
tau_R = 70*ms  # Constante de tiempo de traza postsináptica
n = 0.01  # Constante de aprendizaje
w_max = 1  # Peso máximo

S_ee = Synapses(G, G, model=eqs_plasticity, on_pre=on_pre, on_post=on_post)
S_ee.connect(condition='i != j')

# Inicialización de pesos sinápticos con una distribución aleatoria
w_init = 0.1  # Peso inicial máximo, ajusta según sea necesario para tu modelo
S_ee.w = 'rand() * w_init'

# Simulación para múltiples documentos
n_documentos = len(tfidf_matrix)  # Número de documentos

# Crear explícitamente un objeto Network
net = Network()
net.add(G)
net.add(I)
net.add(S_ei)
net.add(S_ie)
net.add(S_ee)

# Ahora, dentro de tu bucle para procesar cada documento
for doc_idx in range(n_documentos):
    rates = tfidf_matrix[doc_idx] * alpha * Hz
    P = PoissonGroup(M, rates=rates)
    S_input = Synapses(P, G, on_pre='g_e += 0.1')
    S_input.connect(j='i')
    
    # Agregar el grupo Poisson y las sinapsis a la red para este documento
    net.add(P)
    net.add(S_input)
    
    # Monitores (recreados para cada documento para evitar MagicError)
    M_spikes = SpikeMonitor(G)
    M_v = StateMonitor(G, 'v', record=True)
    
    # Agregar los monitores a la red
    net.add(M_spikes)
    net.add(M_v)
    
    # Ejecutar la simulación para este documento
    net.run(600*ms)

    # Visualización
    figure(figsize=(12, 6))
    subplot(121)
    plot(M_spikes.t/ms, M_spikes.i, '.k')
    xlim(0, 600)
    xlabel('Time (ms)')
    ylabel('Neuron index')
    
    subplot(122)
    plot(M_v.t/ms, M_v.u[0]/mV)
    xlabel('Time (ms)')
    ylabel('Membrane potential (mV)')
    
    show()

    # Importante: borra los monitores y el grupo Poisson de la red antes de la próxima iteración
    net.remove(M_spikes)
    net.remove(M_v)
    net.remove(P)
    net.remove(S_input)

: 